In [1]:
#Input
import torch
import torch.nn as nn
import torch.optim as optim

# Define the Generator
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)
    
# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)
    
# Generate synthetic data for training
torch.manual_seed(42)
real_data = torch.rand(100, 1) * 2 - 1  # 100 samples in the range [-1, 1]

# Initialize models, loss, and optimizers
latent_dim = 10
data_dim = 1
G = Generator(latent_dim, data_dim)
D = Discriminator(data_dim)

criterion = nn.BCELoss()
optimizer_G = optim.Adam(G.parameters(), lr=0.001)
optimizer_D = optim.Adam(D.parameters(), lr=0.001)

# Training loop
epochs = 1000
for epoch in range(epochs):
    # Train Discriminator
    latent_samples = torch.randn(real_data.size(0), latent_dim)
    fake_data = G(latent_samples).detach()
    real_labels = torch.ones(real_data.size(0), 1)
    fake_labels = torch.zeros(real_data.size(0), 1)

    optimizer_D.zero_grad()
    real_loss = criterion(D(real_data), real_labels)
    fake_loss = criterion(D(fake_data), fake_labels)
    loss_D = real_loss + fake_loss
    loss_D.backward()
    optimizer_D.step()

    # Train Generator
    latent_samples = torch.randn(real_data.size(0), latent_dim)
    fake_data = G(latent_samples)
    optimizer_G.zero_grad()
    loss_G = criterion(D(fake_data), real_labels)
    loss_G.backward()
    optimizer_G.step()

    # Log progress every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}] - Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}")

# Generate new samples with the trained Generator
latent_samples = torch.randn(5, latent_dim)
with torch.no_grad():
    generated_data = G(latent_samples)
    print(f"Generated data: {generated_data.tolist()}")

Epoch [100/1000] - Loss D: 0.7453, Loss G: 1.3754
Epoch [200/1000] - Loss D: 0.9934, Loss G: 0.9999
Epoch [300/1000] - Loss D: 1.5029, Loss G: 0.7750
Epoch [400/1000] - Loss D: 1.5476, Loss G: 0.6356
Epoch [500/1000] - Loss D: 1.1889, Loss G: 0.9551
Epoch [600/1000] - Loss D: 1.3865, Loss G: 0.7122
Epoch [700/1000] - Loss D: 1.3901, Loss G: 0.6709
Epoch [800/1000] - Loss D: 1.3918, Loss G: 0.7005
Epoch [900/1000] - Loss D: 1.3791, Loss G: 0.7079
Epoch [1000/1000] - Loss D: 1.3745, Loss G: 0.7019
Generated data: [[-0.8730744123458862], [0.3686343729496002], [0.3342606723308563], [0.3323465585708618], [-0.5318933725357056]]


In [2]:
#Strong LLM
import jax
import jax.numpy as jnp
from flax import linen as nn
import optax
import numpy as np


class Generator(nn.Module):
    latent_dim: int
    output_dim: int

    @nn.compact
    def __call__(self, z):
        # z shape: (batch, latent_dim)
        x = nn.Dense(128)(z)
        x = nn.relu(x)
        x = nn.Dense(256)(x)
        x = nn.relu(x)
        x = nn.Dense(self.output_dim)(x)
        x = nn.tanh(x)
        return x

class Discriminator(nn.Module):
    input_dim: int

    @nn.compact
    def __call__(self, x):
        # x shape: (batch, input_dim)
        x = nn.Dense(256)(x)
        x = nn.leaky_relu(x, negative_slope=0.2)
        x = nn.Dense(128)(x)
        x = nn.leaky_relu(x, negative_slope=0.2)
        x = nn.Dense(1)(x)
        x = nn.sigmoid(x)
        return x


def bce_loss(predictions, targets):
    eps = 1e-7  # small constant for numerical stability
    return -jnp.mean(targets * jnp.log(predictions + eps) + (1 - targets) * jnp.log(1 - predictions + eps))


def main():
    # Set up PRNG keys
    key = jax.random.PRNGKey(42)

    # Generate synthetic real data:
    # 100 samples in the range [-1, 1] with 1 feature
    num_samples = 100
    data_dim = 1
    key, subkey = jax.random.split(key)
    real_data = jax.random.uniform(subkey, shape=(num_samples, data_dim), minval=-1, maxval=1)

    # Define latent space dimension
    latent_dim = 10

    # Instantiate the models
    gen_model = Generator(latent_dim=latent_dim, output_dim=data_dim)
    disc_model = Discriminator(input_dim=data_dim)

    # Initialize model parameters using dummy inputs.
    key, subkey = jax.random.split(key)
    gen_params = gen_model.init(subkey, jnp.ones((1, latent_dim)))
    key, subkey = jax.random.split(key)
    disc_params = disc_model.init(subkey, jnp.ones((1, data_dim)))

    # Set up optimizers for both models using Adam
    gen_optimizer = optax.adam(learning_rate=0.001)
    disc_optimizer = optax.adam(learning_rate=0.001)
    gen_opt_state = gen_optimizer.init(gen_params)
    disc_opt_state = disc_optimizer.init(disc_params)

    epochs = 1000

    # Define a jitted discriminator training step.
    @jax.jit
    def disc_train_step(disc_params, disc_opt_state, gen_params, real_data, key):
        batch_size = real_data.shape[0]
        key, subkey = jax.random.split(key)
        # Sample latent vectors from a normal distribution
        latent_samples = jax.random.normal(subkey, shape=(batch_size, latent_dim))
        # Generate fake data with the current Generator (stop gradient so gradients don't flow into Generator)
        fake_data = jax.lax.stop_gradient(gen_model.apply(gen_params, latent_samples))
        real_labels = jnp.ones((batch_size, 1))
        fake_labels = jnp.zeros((batch_size, 1))

        def loss_disc_fn(params):
            real_pred = disc_model.apply(params, real_data)
            fake_pred = disc_model.apply(params, fake_data)
            return bce_loss(real_pred, real_labels) + bce_loss(fake_pred, fake_labels)

        loss_D, grads = jax.value_and_grad(loss_disc_fn)(disc_params)
        updates, disc_opt_state = disc_optimizer.update(grads, disc_opt_state)
        disc_params = optax.apply_updates(disc_params, updates)
        return disc_params, disc_opt_state, loss_D, key

    # Define a jitted generator training step.
    @jax.jit
    def gen_train_step(gen_params, gen_opt_state, disc_params, batch_size, key):
        key, subkey = jax.random.split(key)
        latent_samples = jax.random.normal(subkey, shape=(batch_size, latent_dim))

        def loss_gen_fn(params):
            fake_data = gen_model.apply(params, latent_samples)
            # Generator tries to fool discriminator so labels are 1.
            pred = disc_model.apply(disc_params, fake_data)
            return bce_loss(pred, jnp.ones((batch_size, 1)))

        loss_G, grads = jax.value_and_grad(loss_gen_fn)(gen_params)
        updates, gen_opt_state = gen_optimizer.update(grads, gen_opt_state)
        gen_params = optax.apply_updates(gen_params, updates)
        return gen_params, gen_opt_state, loss_G, key

    # Training loop
    for epoch in range(epochs):
        # Update Discriminator
        disc_params, disc_opt_state, loss_D, key = disc_train_step(
            disc_params, disc_opt_state, gen_params, real_data, key
        )
        # Update Generator
        gen_params, gen_opt_state, loss_G, key = gen_train_step(
            gen_params, gen_opt_state, disc_params, real_data.shape[0], key
        )
        # Log progress every 100 epochs
        if (epoch + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{epochs}] - Loss D: {loss_D:.4f}, Loss G: {loss_G:.4f}")

    # Generate new samples with the trained Generator
    key, subkey = jax.random.split(key)
    latent_samples = jax.random.normal(subkey, shape=(5, latent_dim))
    generated_data = gen_model.apply(gen_params, latent_samples)
    print("Generated data:", np.array(generated_data).tolist())

if __name__ == "__main__":
    main()


TypeError: Shapes must be 1D sequences of concrete values of integer type, got (Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>, 10).
If using `jit`, try using `static_argnums` or applying `jit` to smaller subfunctions.
The error occurred while tracing the function gen_train_step at C:\Users\15157\AppData\Local\Temp\ipykernel_51176\1738399393.py:99 for jit. This concrete value was not available in Python because it depends on the value of the argument batch_size.

In [3]:
#Weak LLM
import jax
from jax import random  # MODIFIED: Cleaned up unused imports
import jax.numpy as jnp  # MODIFIED: Ensure consistent import of jax.numpy as jnp
# from flax import linen as nn  # Commented out unused import
# import optax  # Commented out unused import

def main():
    """
    Main function to execute the training and generation of samples.

    This function initializes the model parameters, trains the Generator (G) 
    and Discriminator (D) models, and generates new samples after training.
    """
    # Initialize model parameters, training configurations, etc.
    key = random.PRNGKey(0)  # Seed for randomness
    latent_dim = 100  # Dimensionality of the latent space
    # Add more initialization code as needed...

    # Example training loop (details omitted for brevity)
    epochs = 1000
    for epoch in range(epochs):
        # Assume loss_D and loss_G are computed here
        loss_D, loss_G = train_step(epoch)  # Placeholder function

        # Log progress every 100 epochs
        if (epoch + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{epochs}] - Loss D: {loss_D:.4f}, Loss G: {loss_G:.4f}")

    # Generate new samples with the trained Generator
    latent_samples = random.normal(key, (5, latent_dim))
    generated_data = G.apply(G_params, latent_samples)
    print(f"Generated data: {generated_data.tolist()}")

def train_step(epoch):
    """
    Placeholder function for training step.
    
    This function is meant to perform a single training step for the 
    Generator and Discriminator models.

    Parameters:
        epoch (int): The current epoch number.

    Returns:
        tuple: A tuple containing the loss for the Discriminator and 
               Generator.
    """
    # Placeholder implementation
    loss_D = jnp.random.rand()  # Random loss for demonstration
    loss_G = jnp.random.rand()  # Random loss for demonstration
    return loss_D, loss_G

if __name__ == "__main__":
    main()

AttributeError: module 'jax.numpy' has no attribute 'random'

In [ ]:
"""
Error Code:
loss_D = jnp.random.rand()  # Random loss for demonstration
loss_G = jnp.random.rand()  # Random loss for demonstration


Error:
module 'jax.numpy' has no attribute 'random'


Fix Guide:
Write the real loss function and calculate the real loss based on the model output


Correct Code:
def bce_loss(predictions, targets):
    bce = - (targets * jnp.log(predictions + 1e-8) + (1 - targets) * jnp.log(1 - predictions + 1e-8))
    return jnp.mean(bce)
"""


"""
Error Code:
# from flax import linen as nn  # Commented out unused import


Error:
The linen module of Flax is not introduced, and the model Generator and Discriminator cannot be defined later


Fix Guide:
Uncomment and correctly import flax.linen as nn to define the neural network module using Flax
Use Flax to define the Generator and Discriminator models, constructing the same fully connected layers and activation functions


Correct Code:
from flax import linen as nn

class Generator(nn.Module):
    latent_dim: int
    output_dim: int

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(128)(x)
        x = nn.relu(x)
        x = nn.Dense(256)(x)
        x = nn.relu(x)
        x = nn.Dense(self.output_dim)(x)
        x = nn.tanh(x)
        return x

class Discriminator(nn.Module):
    input_dim: int

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(256)(x)
        x = nn.leaky_relu(x, negative_slope=0.2)
        x = nn.Dense(128)(x)
        x = nn.leaky_relu(x, negative_slope=0.2)
        x = nn.Dense(1)(x)
        x = nn.sigmoid(x)
        return x
"""


"""
Error Code:
generated_data = G.apply(G_params, latent_samples)


Error:
The Generator parameters are not initialized in the code, resulting in G_params being undefined


Fix Guide:
Before calling G.apply, use G.init to initialize the model parameters based on an example input and save the result to G_params


Correct Code:
key, subkey = random.split(key)
G_params = G.init(subkey, jnp.ones((1, latent_dim)))
"""


"""
Error Code:
def train_step(epoch):
    # Placeholder implementation
    loss_D = jnp.random.rand()  # Random loss for demonstration
    loss_G = jnp.random.rand()  # Random loss for demonstration
    return loss_D, loss_G


Error:
The training step does not implement the actual training steps of Generator and Discriminator, actual forward propagation, loss calculation, gradient derivation and parameter update logic


Fix Guide:
Write a complete train_step function:
Use the generator to generate fake samples
Calculate the discriminator loss on real samples and fake samples
Calculate the discriminator gradient and update the discriminator parameters
Calculate the generator loss and update the generator parameters
Use jax.value_and_grad and optax to complete the parameter update


Correct Code:
def train_step(G_params, D_params, G_opt_state, D_opt_state, real_data, key, latent_dim, G, D, G_optimizer, D_optimizer):
    key, subkey = random.split(key)
    latent_samples = random.normal(subkey, (real_data.shape[0], latent_dim))
    fake_data = G.apply(G_params, latent_samples)
    
    real_labels = jnp.ones((real_data.shape[0], 1))
    fake_labels = jnp.zeros((real_data.shape[0], 1))
    
    def d_loss_fn(D_params):
        real_logits = D.apply(D_params, real_data)
        fake_logits = D.apply(D_params, fake_data)
        real_loss = bce_loss(real_logits, real_labels)
        fake_loss = bce_loss(fake_logits, fake_labels)
        loss = real_loss + fake_loss
        return loss

    d_loss, d_grads = value_and_grad(d_loss_fn)(D_params)
    D_updates, D_opt_state = D_optimizer.update(d_grads, D_opt_state, D_params)
    D_params = optax.apply_updates(D_params, D_updates)
    
    key, subkey = random.split(key)
    latent_samples = random.normal(subkey, (real_data.shape[0], latent_dim))
    
    def g_loss_fn(G_params):
        fake_data = G.apply(G_params, latent_samples)
        logits = D.apply(D_params, fake_data)
        loss = bce_loss(logits, real_labels) 
        return loss

    g_loss, g_grads = value_and_grad(g_loss_fn)(G_params)
    G_updates, G_opt_state = G_optimizer.update(g_grads, G_opt_state, G_params)
    G_params = optax.apply_updates(G_params, G_updates)
    
    return G_params, D_params, G_opt_state, D_opt_state, d_loss, g_loss, key
"""


"""
Error Code:
latent_dim = 100  # Dimensionality of the latent space


Error:
Inconsistent values ​​of latent_dim compared to PyTorch code


Fix Guide:
Change latent_dim to 10


Correct Code:
latent_dim = 10
"""


"""
Error Code:
key = random.PRNGKey(0)  # Seed for randomness


Error:
100 samples in the range [-1, 1] are generated in the PyTorch code, but this real data is not generated in the JAX code


Fix Guide:
Use random.uniform to generate real data with shape (100, 1) and range [-1, 1]


Correct Code:
key = random.PRNGKey(0)
key, subkey = random.split(key)
real_data = random.uniform(subkey, (100, 1), minval=-1, maxval=1)
"""


"""
Error Code:
key = random.PRNGKey(0)
key, subkey = random.split(key)
real_data = random.uniform(subkey, (100, 1), minval=-1, maxval=1)
latent_dim = 10  # Dimensionality of the latent space

key, subkey = random.split(key)
real_data = random.uniform(subkey, (100, 1), minval=-1, maxval=1)

Error:
The Generator and Discriminator, as well as the random model parameters and optimizer, are not initialized in the main() function.


Fix Guide:
Initialize the model Generator and Discriminato
Initialize the model parameters
Use optax to initialize the optimizer


Correct Code:
import optax

key = random.PRNGKey(0)
key, subkey = random.split(key)
real_data = random.uniform(subkey, (100, data_dim), minval=-1, maxval=1)

G = Generator(latent_dim=latent_dim, output_dim=data_dim)
D = Discriminator(input_dim=data_dim)

key, subkey = random.split(key)
G_params = G.init(subkey, jnp.ones((1, latent_dim)))
key, subkey = random.split(key)
D_params = D.init(subkey, jnp.ones((1, data_dim)))

G_optimizer = optax.adam(learning_rate=0.001)
D_optimizer = optax.adam(learning_rate=0.001)
G_opt_state = G_optimizer.init(G_params)
D_opt_state = D_optimizer.init(D_params)
"""


"""
Error Code:
loss_D, loss_G = train_step(epoch)


Error:
train_step() missing 10 required positional arguments: 'D_params', 'G_opt_state', 'D_opt_state', 'real_data', 'key', 'latent_dim', 'G', 'D', 'G_optimizer', and 'D_optimizer'


Fix Guide:
Add the parameters required by train_step()


Correct Code:
G_params, D_params, G_opt_state, D_opt_state, d_loss, g_loss, key = train_step(
    G_params, D_params, G_opt_state, D_opt_state, real_data, key, latent_dim, G, D, G_optimizer, D_optimizer
)
"""


"""
Error Code:
import jax
from jax import random  # MODIFIED: Cleaned up unused imports
import jax.numpy as jnp  # MODIFIED: Ensure consistent import of jax.numpy as jnp
from flax import linen as nn
# import optax  # Commented out unused import


Error:
name 'value_and_grad' is not defined


Fix Guide:
From jax import value_and_grad


Correct Code:
import jax
from jax import random, value_and_grad  # MODIFIED: Cleaned up unused imports
import jax.numpy as jnp  # MODIFIED: Ensure consistent import of jax.numpy as jnp
from flax import linen as nn
# import optax  # Commented out unused import

"""


"""
Error Code:
key, subkey = random.split(key)
real_data = random.uniform(subkey, (100, data_dim), minval=-1, maxval=1)


Error:
Undefined variable data_dim


Fix Guide:
Define the data_dim variable before using it


Correct Code:
data_dim = 1
key, subkey = random.split(key)
real_data = random.uniform(subkey, (100, data_dim), minval=-1, maxval=1)
"""


"""
Error Code:
if (epoch + 1) % 100 == 0:
    print(f"Epoch [{epoch + 1}/{epochs}] - Loss D: {loss_D:.4f}, Loss G: {loss_G:.4f}")


Error:
The variables loss_D and loss_G are used when printing, but the names of the loss variables returned during the training step are d_loss and g_loss respectively


Fix Guide:
Replace loss_D and loss_G in the print statements with the correct variable names d_loss and g_loss


Correct Code:
if (epoch + 1) % 100 == 0:
    print(f"Epoch [{epoch + 1}/{epochs}] - Loss D: {d_loss:.4f}, Loss G: {g_loss:.4f}")

"""


"""
Error Code:
key, subkey = random.split(key)
real_data = random.uniform(subkey, (100, 1), minval=-1, maxval=1)
latent_dim = 10  # Dimensionality of the latent space
# Add more initialization code as needed...
    
data_dim = 1
key, subkey = random.split(key)
real_data = random.uniform(subkey, (100, data_dim), minval=-1, maxval=1)


Error:
The first initialization of real_data is redundant


Fix Guide:
Remove redundant first initialization


Correct Code:
latent_dim = 10  # Dimensionality of the latent space
data_dim = 1     # Dimensionality of the data

key, subkey = random.split(key)
real_data = random.uniform(subkey, (100, data_dim), minval=-1, maxval=1)
"""


In [11]:
#Fixed Code
import jax
from jax import random, value_and_grad  # MODIFIED: Cleaned up unused imports
import jax.numpy as jnp  # MODIFIED: Ensure consistent import of jax.numpy as jnp
from flax import linen as nn
import optax  # Commented out unused import


class Generator(nn.Module):
    latent_dim: int
    output_dim: int

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(128)(x)
        x = nn.relu(x)
        x = nn.Dense(256)(x)
        x = nn.relu(x)
        x = nn.Dense(self.output_dim)(x)
        x = nn.tanh(x)
        return x

class Discriminator(nn.Module):
    input_dim: int

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(256)(x)
        x = nn.leaky_relu(x, negative_slope=0.2)
        x = nn.Dense(128)(x)
        x = nn.leaky_relu(x, negative_slope=0.2)
        x = nn.Dense(1)(x)
        x = nn.sigmoid(x)
        return x

def bce_loss(predictions, targets):
    bce = - (targets * jnp.log(predictions + 1e-8) + (1 - targets) * jnp.log(1 - predictions + 1e-8))
    return jnp.mean(bce)

def train_step(G_params, D_params, G_opt_state, D_opt_state, real_data, key, latent_dim, G, D, G_optimizer, D_optimizer):
    key, subkey = random.split(key)
    latent_samples = random.normal(subkey, (real_data.shape[0], latent_dim))
    fake_data = G.apply(G_params, latent_samples)
    
    real_labels = jnp.ones((real_data.shape[0], 1))
    fake_labels = jnp.zeros((real_data.shape[0], 1))
    
    def d_loss_fn(D_params):
        real_logits = D.apply(D_params, real_data)
        fake_logits = D.apply(D_params, fake_data)
        real_loss = bce_loss(real_logits, real_labels)
        fake_loss = bce_loss(fake_logits, fake_labels)
        loss = real_loss + fake_loss
        return loss

    d_loss, d_grads = value_and_grad(d_loss_fn)(D_params)
    D_updates, D_opt_state = D_optimizer.update(d_grads, D_opt_state, D_params)
    D_params = optax.apply_updates(D_params, D_updates)
    
    key, subkey = random.split(key)
    latent_samples = random.normal(subkey, (real_data.shape[0], latent_dim))
    
    def g_loss_fn(G_params):
        fake_data = G.apply(G_params, latent_samples)
        logits = D.apply(D_params, fake_data)
        loss = bce_loss(logits, real_labels) 
        return loss

    g_loss, g_grads = value_and_grad(g_loss_fn)(G_params)
    G_updates, G_opt_state = G_optimizer.update(g_grads, G_opt_state, G_params)
    G_params = optax.apply_updates(G_params, G_updates)
    
    return G_params, D_params, G_opt_state, D_opt_state, d_loss, g_loss, key

def main():
    """
    Main function to execute the training and generation of samples.

    This function initializes the model parameters, trains the Generator (G) 
    and Discriminator (D) models, and generates new samples after training.
    """
    # Initialize model parameters, training configurations, etc.
    key = random.PRNGKey(0)
    latent_dim = 10  # Dimensionality of the latent space
    data_dim = 1     # Dimensionality of the data

    key, subkey = random.split(key)
    real_data = random.uniform(subkey, (100, data_dim), minval=-1, maxval=1)

    G = Generator(latent_dim=latent_dim, output_dim=data_dim)
    D = Discriminator(input_dim=data_dim)

    key, subkey = random.split(key)
    G_params = G.init(subkey, jnp.ones((1, latent_dim)))
    key, subkey = random.split(key)
    D_params = D.init(subkey, jnp.ones((1, data_dim)))

    G_optimizer = optax.adam(learning_rate=0.001)
    D_optimizer = optax.adam(learning_rate=0.001)
    G_opt_state = G_optimizer.init(G_params)
    D_opt_state = D_optimizer.init(D_params)

    # Example training loop (details omitted for brevity)
    epochs = 1000
    for epoch in range(epochs):
        G_params, D_params, G_opt_state, D_opt_state, d_loss, g_loss, key = train_step(
            G_params, D_params, G_opt_state, D_opt_state, real_data, key, latent_dim, G, D, G_optimizer, D_optimizer
        )

        # Log progress every 100 epochs
        if (epoch + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{epochs}] - Loss D: {d_loss:.4f}, Loss G: {g_loss:.4f}")


    # Generate new samples with the trained Generator
    latent_samples = random.normal(key, (5, latent_dim))
    generated_data = G.apply(G_params, latent_samples)
    print(f"Generated data: {generated_data.tolist()}")
    

if __name__ == "__main__":
    main()

Epoch [100/1000] - Loss D: 0.6193, Loss G: 1.9655
Epoch [200/1000] - Loss D: 0.9221, Loss G: 1.2601
Epoch [300/1000] - Loss D: 1.1314, Loss G: 0.6061
Epoch [400/1000] - Loss D: 1.1258, Loss G: 0.9030
Epoch [500/1000] - Loss D: 1.4447, Loss G: 0.7525
Epoch [600/1000] - Loss D: 1.4007, Loss G: 0.6654
Epoch [700/1000] - Loss D: 1.3958, Loss G: 0.6561
Epoch [800/1000] - Loss D: 1.3836, Loss G: 0.6977
Epoch [900/1000] - Loss D: 1.3804, Loss G: 0.6840
Epoch [1000/1000] - Loss D: 1.3873, Loss G: 0.7019
Generated data: [[0.794902503490448], [-0.07745052129030228], [0.49759846925735474], [0.11663448065519333], [0.5132001042366028]]
